In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = 'wp.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8', errors="ignore")
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1538371 characters


In [4]:
print(text[:500])

﻿
Замысел

В 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге восходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подписано: «25 июля 1826 года». Том помечен днем казни декабристов.

Над заглавием в тучах звезда.

Полярная.

Объявление было целым манифестом. Герцен говорил о восстании декабристов и о севастопольской кампании; спрашивал, неужели «севастопольский солдат, израненный и твердый, как гранит, испытавший св


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

153 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

text_as_int, text, len(text_as_int), len(text)

### train and target

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

﻿


З
а
м


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\ufeff\nЗамысел\n\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге во'
'сходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подписа'
'но: «25 июля 1826 года». Том помечен днем казни декабристов.\n\nНад заглавием в тучах звезда.\n\nПолярная'
'.\n\nОбъявление было целым манифестом. Герцен говорил о восстании декабристов и о севастопольской кампа'
'нии; спрашивал, неужели «севастопольский солдат, израненный и твердый, как гранит, испытавший свою си'


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\ufeff\nЗамысел\n\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге в'
Target data: '\nЗамысел\n\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге во'


In [12]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),                                   
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(vocab_size)
        
    ])
    return model

In [15]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 153) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           19584     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 512)           524800    
_________________________________________________________________
dropout (Dropout)            (64, None, 512)           0

In [18]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 153), dtype=float32, numpy=
array([[ 8.6729005e-06,  6.2254244e-06,  1.1632470e-05, ...,
        -9.0733238e-06,  4.6823275e-06,  4.2760880e-07],
       [ 1.4790306e-05,  1.8809798e-05,  2.5416206e-05, ...,
        -2.7209924e-05,  1.2460094e-05,  3.8511107e-06],
       [ 1.5797532e-05,  3.6751360e-05,  3.5946774e-05, ...,
        -3.6901525e-05,  2.3689643e-05,  1.2601597e-06],
       ...,
       [ 4.1585820e-04,  7.6316419e-04, -6.8091741e-04, ...,
        -1.6209480e-04, -1.9294856e-03,  2.8319447e-04],
       [ 3.9302971e-04,  7.9576654e-04, -6.7362306e-04, ...,
        -1.2666169e-04, -1.8632745e-03,  2.7227832e-04],
       [ 3.5701875e-04,  8.6884457e-04, -6.1404979e-04, ...,
        -9.0593370e-05, -1.7989919e-03,  2.4436213e-04]], dtype=float32)>

In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'азывал, что она знает, что делает, перешла через всю комнату к Пьеру и подала ему свечу. Он зажег ее'

Next Char Predictions: 
 'ЕJhb[лNNйезGж…д„а!В&&,—9дхaСmGШ8зV—НГ,Еg6D?ы#дюшц–![ОGГfР\n2ПеасйЕ“ыюРТ:.uNмЗ6xK<ZЭIc.ЯEЗ\ufeff&Щ4м“>t6О2в'


## Train the model

In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 153)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.030456


In [22]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [23]:
!rm -rf ./training_checkpoints

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [24]:
!ls ./training_checkpoints

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [26]:
EPOCHS = 100

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
475/475 [==============================] - 775s 2s/step - loss: 2.8801
Epoch 2/100
475/475 [==============================] - 912s 2s/step - loss: 1.8478
Epoch 3/100
475/475 [==============================] - 870s 2s/step - loss: 1.5284
Epoch 4/100
475/475 [==============================] - 492s 1s/step - loss: 1.4071
Epoch 5/100
475/475 [==============================] - 299s 627ms/step - loss: 1.3252
Epoch 6/100
475/475 [==============================] - 354s 744ms/step - loss: 1.2586
Epoch 7/100
475/475 [==============================] - 358s 752ms/step - loss: 1.1959
Epoch 8/100
475/475 [==============================] - 337s 707ms/step - loss: 1.1346
Epoch 9/100
475/475 [==============================] - 345s 724ms/step - loss: 1.0724
Epoch 10/100
475/475 [==============================] - 270s 566ms/step - loss: 1.0087
Epoch 11/100
475/475 [==============================] - 339s 712ms/step - loss: 0.9438
Epoch 12/100
475/475 [==============================] - 329s 691

## Generate text

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_100'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            19584     
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 1024)           4722688   
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_2 (Dense)              (1, None, 512)            524800    
_________________________________________________________________
dropout_1 (Dropout)          (1, None, 512)           

In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [32]:
text_ = generate_text(model, start_string=u"Анатоль вышел из комнаты и через несколько минут вернулся  ")
print(text_)

Анатоль вышел из комнаты и через несколько минут вернулся  у ее белого ружейного ответа, вовсе не на середине грязной дороги.

Засветились огни, и слышнее стал говор. Капитан Тушин, распорядившись по воле и начала будущего мальчика. Только что надеть в десяти шагах не только для того, чтобы принять участие в этой войне, во всем ужасе представилась его вина и позор в том, что я видела и которую никогда не забуду. Мне кажется, что могу исполнять ваше сожаление? А для себя, когда я сделал ему знак рыцарей Востока и Иерусалима, он ответил мне тем же и с кро


In [33]:
len(text_)

559

In [34]:
text_ = generate_text(model, start_string=u"Долохов презрительно и снисходительно улыбался ")
print(text_)

Долохов презрительно и снисходительно улыбался и что все будет хорошо поэтические люди? То все кончено и вмоде ко мне с фружтовым приастарела верно, и я не способен идти sur le canap&#233; bleu, le comte Денисов скинул обе подушки на пол. Кошелька не было.

— Вы не видали моего самого в что сын в киев отдельных ступеней, и как предмет для меня чем-нибудь, кроме как другие могли не поверить ему в лесу и с одними дурными, не значать, не просыпаясь. На другой день он проснулся весь и не был так умен. Вдруг короля и засмеялся.

— Parlez-moi de &


In [ ]:
len(text_)